In [ ]:
#Reads in two excel files, compare values between the two files based on unique tag/key name (key_column), outputs only changes in file 
#with corresponding row number

#Need to put script and two files in the same folder - can build out to include actual file path but kept it simple for now
#For Excel with multiple sheets, need to specify sheet name (or if only one sheet can either specify sheet_name or remove)

import pandas as pd

#Compare Function
def compare_excel_files(file1, file2, output_file, key_column):
    
    #Read the two excel files and particular sheet
    #############Specify Sheet names here ##############
    df1 = pd.read_excel(file1, sheet_name="Sheet1", dtype= str)
    df2 = pd.read_excel(file2, sheet_name="AF Specification", dtype= str)

    #Add row number column for reference to track which row changes are made
    df1.insert(0, "Row_Number", df1.index + 2)
    df2.insert(0, "Row_Number", df2.index + 2)
    
    #Set key column as the index for comparison
    df1.set_index(key_column, inplace=True)
    df2.set_index(key_column, inplace=True)

    #Reindex to match between the two files
    df2 = df2.reindex(index=df1.index, columns = df1.columns)

    #Create Dataframe to store differences
    diff_df = pd.DataFrame(columns=["Row Number"] + list(df1.columns))

    #Compare values based on Tag name (key_column)
    for name in df1.index:
        row_diff = {"Row_Number": df1.at [name, "Row_Number"]}
        has_difference = False
        
        for col in df1.columns:
            if col == "Row_Number":
                continue
        
            val1 = df1.at[name,col] if name in df1.index else None
            val2 = df2.at[name,col] if name in df2.index else None
    
            #Ignore if both are Nan/blank
            if pd.isna(val1) and pd.isna(val2):
                row_diff[col] = val1
                continue

            #Mark the Differences
            if val1 != val2:
                row_diff[col] = f"{val1} <-- {val2}"
                has_difference = True
            else:
                row_diff[col] = val1
        
        if has_difference:
            diff_df = pd.concat([diff_df, pd.DataFrame([row_diff], index=[name])])

    #Reset Index
    diff_df.reset_index(inplace=True)

    #Save Differences
    if not diff_df.empty:
        diff_df.drop('Row Number', axis='columns', inplace=True)
        diff_df.to_excel(output_file, index = False)
        print("Differences saved as {output_file}")
    else:
        print ("No Differences Found")

#########Specify Files Here ##################
#Use - will output changes based between the two files
#First File - new file
#Second File - old file
#Differences - name of output file with differences
#key_column = enter name of column that has unique value for comparison
compare_excel_files("V2.xlsx", "V1.xlsx", "Differences.xlsx", key_column = "Name")
